In [1]:
import numpy as np
import pandas as pd
import plotly

%matplotlib inline
import geopandas 
import matplotlib.pyplot as plt 
from geopy.geocoders import Nominatim
from geopy.geocoders import ArcGIS
import folium
from folium.plugins import MarkerCluster

In [4]:
by_major_city = pd.read_csv('GlobalLandTemperaturesByMajorCity.csv')
by_major_city = by_major_city.drop_duplicates (['City'])
by_major_city.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1849-01-01,26.704,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W
1977,1850-01-01,15.986,1.537,Addis Abeba,Ethiopia,8.84N,38.11E
3942,1796-01-01,19.649,2.286,Ahmadabad,India,23.31N,72.52E
6555,1791-05-01,20.836,1.993,Aleppo,Syria,36.17N,37.79E
9224,1791-05-01,20.772,1.848,Alexandria,Egypt,31.35N,30.16E


In [5]:
geolocator = ArcGIS(user_agent = "myGeofriend")
LAT = []
LONG = []
for city in by_major_city.City.tolist():
    location = geolocator.geocode(city)
    LAT.append(location.latitude)
    LONG.append(location.longitude)

In [7]:
by_major_city['Latitude'] = LAT
by_major_city['Longitude'] = LONG

In [65]:
color_mapping = {
    'blue': lambda t: t < 10,
    'green': lambda t: 10 <= t < 15,
    'red': lambda t: t >= 15
}

In [75]:
world_map = folium.Map(location=[0, 0], zoom_start=2)

In [76]:
geolocator = ArcGIS(user_agent="myGeofriend")
marker_cluster = MarkerCluster().add_to(world_map)

In [77]:
for i in range(len(by_major_city)):
        lat = by_major_city.iloc[i]['Latitude']
        long = by_major_city.iloc[i]['Longitude']
        temp = by_major_city.iloc[i]['AverageTemperature']
        for color, condition in color_mapping.items():
             if condition(temp):
                marker_color = color
                break
        folium.Marker(
        location = [lat, long],
        popup = f"Temperature: {temp}°C",
        icon = folium.Icon(color = marker_color, icon ='info-sign')
    ).add_to(marker_cluster)

In [78]:
world_map

In [55]:
world_map= folium.Map()
geolocator = ArcGIS(user_agent="myGeofriend")
marker_cluster = MarkerCluster().add_to(world_map)

for i in range(len(by_major_city)):
        lat = by_major_city.iloc[i]['Latitude']
        long = by_major_city.iloc[i]['Longitude']
        radius = 5
        folium.CircleMarker(location = [lat, long], radius = radius,fill = True, color = marker_color).add_to(marker_cluster)

In [9]:
import folium
from folium.plugins import MarkerCluster, TimestampedGeoJson
from geopy.geocoders import ArcGIS

In [12]:
world_map = folium.Map()
geolocator = ArcGIS(user_agent="myGeofriend")
marker_cluster = MarkerCluster().add_to(world_map)

In [57]:
color_mapping = {
    'blue': lambda t: t < 10,
    'green': lambda t: 10 <= t < 15,
    'red': lambda t: t >= 15
}

In [58]:
marker_cluster = MarkerCluster().add_to(world_map)

In [59]:
for i in range(len(by_major_city)):
        lat = by_major_city.iloc[i]['Latitude']
        long = by_major_city.iloc[i]['Longitude']
        radius = 5
        folium.CircleMarker(location = [lat, long], radius = radius,fill = True, color = color_mapping).add_to(marker_cluster)

In [61]:
gdf = geopandas.GeoDataFrame(
    by_major_city,  
    geometry = geopandas.points_from_xy(by_major_city.Longitude, by_major_city.Latitude)
)

In [62]:
geojson_data = gdf.__geo_interface__

In [63]:
world_map = folium.Map()
folium.Choropleth(
    geo_data = geojson_data, 
    data = by_major_city,
    columns = ['City', 'AverageTemperature'],
    key_on ='feature.properties.City', 
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Temperature'
).add_to(world_map)

In [64]:
world_map